In [1]:
import pandas as pd
import numpy as np

In [2]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

### Data

In [3]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

In [4]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [5]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).as_matrix()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [6]:

def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).as_matrix()

In [7]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [8]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)

In [9]:
# Configuring a session
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

In [10]:
# Import Keras
from keras import backend as K
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

Using TensorFlow backend.


In [11]:
# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout
from keras.layers.normalization import BatchNormalization

In [12]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32

In [13]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [14]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

C:\Users\hbandoli\Anaconda3.0\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':


In [15]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


- Defining the Architecture of LSTM

In [16]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 32)                5376      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [18]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Instructions for updating:
Use tf.cast instead.
Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 33s 4ms/step - loss: 1.3233 - acc: 0.4332 - val_loss: 1.1553 - val_acc: 0.4649
Epoch 2/30
7352/7352 [==============================] - 30s 4ms/step - loss: 1.0250 - acc: 0.5536 - val_loss: 1.0156 - val_acc: 0.5806: 1.0503 - acc: 0.5 - E
Epoch 3/30
7352/7352 [==============================] - 29s 4ms/step - loss: 0.8296 - acc: 0.6432 - val_loss: 0.7981 - val_acc: 0.6043
Epoch 4/30
7352/7352 [==============================] - 29s 4ms/step - loss: 0.7209 - acc: 0.6542 - val_loss: 0.7564 - val_acc: 0.59557262 -  - ETA: 2s - loss: 0.7 - ETA: 1s - l - ETA: 0s - loss: 0.7219 - acc: 
Epoch 5/30
7352/7352 [==============================] - 29s 4ms/step - loss: 0.6490 - acc: 0.6702 - val_loss: 0.7268 - val_acc: 0.6237 1s - lo
Epoch 6/30
7352/7352 [==============================] - 29s 4ms/step - loss: 0.6264 - acc: 0.6780 - val_loss: 0.7316 - val

In [19]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 1s 316us/step


In [20]:
score

[0.5226698747499483, 0.8914149983033594]

- With a simple 2 layer architecture we got 90.09% accuracy and a loss of 0.30
- We can further imporve the performace with Hyperparameter tuning

<h2> Changing dropout rate to 0.2</h2>

In [22]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32

In [23]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.2))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 32)                5376      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [24]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [25]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 69s 9ms/step - loss: 1.1882 - acc: 0.5069 - val_loss: 1.0581 - val_acc: 0.5002
Epoch 2/30
7352/7352 [==============================] - 67s 9ms/step - loss: 0.8497 - acc: 0.6107 - val_loss: 0.7914 - val_acc: 0.6047
Epoch 3/30
7352/7352 [==============================] - 67s 9ms/step - loss: 0.7486 - acc: 0.6391 - val_loss: 0.8275 - val_acc: 0.5711
Epoch 4/30
7352/7352 [==============================] - 66s 9ms/step - loss: 0.7175 - acc: 0.6442 - val_loss: 0.7364 - val_acc: 0.6193
Epoch 5/30
7352/7352 [==============================] - 65s 9ms/step - loss: 0.6340 - acc: 0.7125 - val_loss: 0.6244 - val_acc: 0.7319
Epoch 6/30
7352/7352 [==============================] - 64s 9ms/step - loss: 0.5213 - acc: 0.7779 - val_loss: 0.5573 - val_acc: 0.7604
Epoch 7/30
7352/7352 [==============================] - 62s 8ms/step - loss: 0.4497 - acc: 0.8187 - val_loss: 0.5333 - val_acc: 0.7788
Epoch 8

In [26]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 510        0        27        0                   0   
SITTING                  0      420        67        3                   0   
STANDING                 0      122       406        4                   0   
WALKING                  0        0         0      452                  21   
WALKING_DOWNSTAIRS       0        0         0        1                 415   
WALKING_UPSTAIRS         0        6         0       10                   1   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            1  
STANDING                           0  
WALKING                           23  
WALKING_DOWNSTAIRS                 4  
WALKING_UPSTAIRS                 454  


In [27]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 2s 805us/step


In [28]:
score

[0.3021738717609319, 0.9015948422124194]

<h1> Model 3 :Adding Batch Normalization Layer</h1>

In [21]:
# Initializing parameters
epochs = 30
batch_size = 32
n_hidden = 128

In [22]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
model.add(BatchNormalization())
# Adding a dropout layer
model.add(Dropout(0.25))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 128)               70656     
_________________________________________________________________
batch_normalization_1 (Batch (None, 128)               512       
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 774       
Total params: 71,942
Trainable params: 71,686
Non-trainable params: 256
_________________________________________________________________


In [23]:

# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [24]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 29s 4ms/step - loss: 0.9843 - acc: 0.5869 - val_loss: 0.9753 - val_acc: 0.6081
Epoch 2/30
7352/7352 [==============================] - 26s 4ms/step - loss: 0.7760 - acc: 0.6416 - val_loss: 0.7488 - val_acc: 0.6366
Epoch 3/30
7352/7352 [==============================] - 27s 4ms/step - loss: 0.6563 - acc: 0.6974 - val_loss: 0.7142 - val_acc: 0.6620
Epoch 4/30
7352/7352 [==============================] - 27s 4ms/step - loss: 0.5443 - acc: 0.7348 - val_loss: 0.6005 - val_acc: 0.7092
Epoch 5/30
7352/7352 [==============================] - 26s 4ms/step - loss: 0.3778 - acc: 0.8512 - val_loss: 0.4356 - val_acc: 0.8609
Epoch 6/30
7352/7352 [==============================] - 26s 4ms/step - loss: 0.2239 - acc: 0.9206 - val_loss: 0.3920 - val_acc: 0.8887
Epoch 7/30
7352/7352 [==============================] - 26s 4ms/step - loss: 0.1938 - acc: 0.9309 - val_loss: 0.3853 - val_acc: 0.8907
Epoch 8

In [25]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 537        0         0        0                   0   
SITTING                  0      414        75        0                   0   
STANDING                 0       80       450        2                   0   
WALKING                  0        0         0      463                   8   
WALKING_DOWNSTAIRS       0        0         0        1                 419   
WALKING_UPSTAIRS         0        0         1        2                   0   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            2  
STANDING                           0  
WALKING                           25  
WALKING_DOWNSTAIRS                 0  
WALKING_UPSTAIRS                 468  


In [26]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 2s 632us/step


In [27]:
score

[0.30697344534226023, 0.9334916864608076]

<h2>Conclusion: I got the accuracy of 93.35% with loss is equal to 0.30</h2>